In [4]:

pip install mediapipe opencv-python


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install mediapipe


Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/50.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/50.8 MB 1.9 MB/s eta 0:00:27
   ---------------------------------------- 0.1/50.8 MB 656.4 kB/s eta 0:01:18
   ---------------------------------------- 0.1/50.8 MB 1.1 MB/s eta 0:00:46
   ---------------------------------------- 0.2/50.8 MB 1.2 MB/s eta 0:00:43
   ---------------------------------------- 0.3/50.8 MB 1.5 MB/s eta 0:00:33
   ---------------------------------------- 0.5/50.8 MB 1.8 MB/s eta 0:00:28
    --------------------------------------- 0.7/50.8 MB 2.3 MB/s eta 0:00:22
    --------------------------------------- 1.1/50.8 MB 2.9 MB/s eta 0:00:18
   - -------------------------------------- 1.8/50.8 MB 4.1 MB/s eta 0:00:12
   - -------------------------------------- 2.5/50.8 MB 5.3 MB/s eta 0:00:10
   -- ------------------------------------- 3.0/50.8 MB 5.9 MB/s eta 0:00:09
   

In [ ]:
import cv2
import mediapipe as mp

# Initialize MediaPipe Hands and Drawing utilities
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

# Set up webcam capture
cap = cv2.VideoCapture(0)

# Initialize Hand Detection model
with mp_hands.Hands(
    max_num_hands=1,  # Detect one hand at a time
    min_detection_confidence=0.7,
    min_tracking_confidence=0.7) as hands:

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        # Flip the frame to avoid mirror effect
        frame = cv2.flip(frame, 1)
        
        # Convert the frame to RGB (required by MediaPipe)
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        rgb_frame.flags.writeable = False
        
        # Process the frame and detect hands
        results = hands.process(rgb_frame)
        
        # Convert the frame back to BGR for OpenCV
        rgb_frame.flags.writeable = True
        frame = cv2.cvtColor(rgb_frame, cv2.COLOR_RGB2BGR)
        
        # Check if any hands are detected
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                # Draw landmarks on the hand
                mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
                
                # Perform finger counting based on landmarks
                fingers = []
                # Thumb
                if hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP].x < hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_IP].x:
                    fingers.append(1)
                else:
                    fingers.append(0)
                # Fingers (Index, Middle, Ring, Pinky)
                for id in range(1, 5):
                    if hand_landmarks.landmark[mp_hands.HandLandmark(id * 4)].y < hand_landmarks.landmark[mp_hands.HandLandmark(id * 4 - 2)].y:
                        fingers.append(1)
                    else:
                        fingers.append(0)
                
                total_fingers = fingers.count(1)
                
                # Display finger count
                cv2.putText(frame, f'Fingers: {total_fingers}', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        
        # Display the output
        cv2.imshow("Hand Gesture Recognition", frame)
        
        # Break the loop when 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

# Release the webcam and close the window
cap.release()
cv2.destroyAllWindows()


C:\Users\anuhy\AppData\Roaming\Python\Python311\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
